In [11]:
# Para web scraping y manipulación de datos
import requests

# -----------------------------------------------------------------------

import pandas as pd # Pandas es una poderosa librería para manipulación y análisis
pd.set_option('display.max_columns', None) 

# -----------------------------------------------------------------------

# Para crear un data frame global (con todos los csv)
import glob 

In [12]:


url = 'https://moviesdatabase.p.rapidapi.com/titles'
headers = {
    "X-RapidAPI-Key": "edd6677254msh669e2e1eff928a2p144973jsn5f6a710112b0",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}
pelis_info = []
# Iteramos sobre los años 2015-2020 por separado
for year in range(2015, 2021):
    # Procesamos las películas de cada género
    for genre in ['Drama', 'Comedy', 'Action', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Thriller']:
        for page in range(1, 5): 
            query_params = {
                "page": page,
                "year": year,
                "genre": genre
            }
            try:
                # Realizamos solicitudes a la API para obtener películas que coincidan con el año y el género dados
                response = requests.get(url, headers=headers, params=query_params)
                # Verificamos si la respuesta es exitosa (código de estado 200)
                if response.status_code == 200:
                    movies_data = response.json()
                    if 'results' in movies_data and len(movies_data['results']) > 0:
                        # Iteramos sobre los resultados de las películas obtenidas
                        for movie in movies_data['results']:
                            # Verificamos si el tipo de título es 'Movie' o 'Short'
                            if movie['titleType']['text'] in ['Movie', 'Short']:
                                # Agregamos la información relevante de cada película a la lista de tuplas pelis_info
                                pelis_info.append((
                                    movie['titleType']['text'],
                                    movie['titleText']['text'],
                                    movie['releaseYear']['year'],
                                    movie['releaseDate']['month'] if movie['releaseDate'] else None,
                                    movie['id']
                                ))
                else:
                    print(f"Error al acceder a la API. Código de estado: {response.status_code}")
            except Exception as error:
                print(f"Error al acceder a la API: {error}")

print("Información de películas obtenida con éxito:")
print(pelis_info)


Información de películas obtenida con éxito:
[('Short', 'Shulie', 2015, 4, 'tt0177220'), ('Movie', 'Snowblind', 2015, None, 'tt0297400'), ('Movie', 'Chatô: O Rei do Brasil', 2015, 11, 'tt0337926'), ('Movie', 'Transeúntes', 2015, 4, 'tt0346045'), ('Movie', 'The Stanford Prison Experiment', 2015, 7, 'tt0420293'), ('Movie', 'High-Rise', 2015, 6, 'tt0462335'), ('Movie', 'The Man Who Knew Infinity', 2015, 7, 'tt0787524'), ('Movie', 'Miles Ahead', 2015, 4, 'tt0790770'), ('Movie', 'The Danish Girl', 2015, 1, 'tt0810819'), ('Movie', 'Chooriyan', 2015, 6, 'tt0977643'), ('Movie', 'Soapte de amor', 2015, 4, 'tt0191476'), ('Movie', 'Transeúntes', 2015, 4, 'tt0346045'), ('Short', 'Prozac tango', 2015, 11, 'tt0420097'), ('Movie', 'Ant-Man', 2015, 7, 'tt0478970'), ('Movie', 'The Wedding Ringer', 2015, 3, 'tt0884732'), ('Short', 'Publicité SAAQ - campagne publicitaire sur les drogues au volant (Pigeon)', 2015, 4, 'tt10005306'), ('Movie', 'Jurassic World', 2015, 6, 'tt0369610'), ('Movie', 'Ant-Man', 20

In [18]:
df = pd.DataFrame(pelis_info, columns=['Tipo', 'Nombre', 'Año', 'Mes', 'Id'])


# Convertir la columna 'Mes' a tipo entero
df['Mes'] = df['Mes'].astype(float).astype('Int64')

# Convertir la columna 'Mes' a tipo objeto
df['Mes'] = df['Mes'].astype('object')

# Rellenar los valores nulos con 'No Info'
df['Mes'] = df['Mes'].fillna('No Info')


df

,Tipo,Nombre,Año,Mes,Id
0,Short,Shulie,2015,4,tt0177220
1,Movie,Snowblind,2015,No Info,tt0297400
2,Movie,Chatô: O Rei do Brasil,2015,11,tt0337926
3,Movie,Transeúntes,2015,4,tt0346045
4,Movie,The Stanford Prison Experiment,2015,7,tt0420293
...,...,...,...,...,...
479,Movie,Antebellum,2020,9,tt10065694
480,Short,32 Goldfish,2020,2,tt10065208
481,Movie,Breaking Surface,2020,2,tt10081762
482,Movie,Range of Sniper,2020,6,tt10082520


In [ ]:
# Usamos el método de Pandas para guardar un DataFrame en un archivo CSV
df.to_csv('datos_api.csv', index=False) # Este argumento le indica a Pandas que no incluya el índice del DataFrame en el archivo CSV

# Creamos un conjunto para almacenar nombres de archivo únicos (no repetimos películas)
archivos_unicos = set()

# Inicializamos una lista vacía para almacenar los DataFrames de cada archivo CSV
dataframes = []

# Encontramos todos los archivos CSV en el directorio actual pasándole a la función glob() del módulo glob el patrón '*.csv'
archivos_csv = glob.glob('*.csv')

# Iteramos sobre cada archivo CSV
for archivo in archivos_csv:
    # Lee el archivo CSV y lo convierte en un DataFrame
    df = pd.read_csv(archivo)
    # Iteramos por pares (indice, fila) sobre el índice de la fila y sobre los datos que esta contiene mediante 'fila'
    for indice, fila in df.iterrows(): # Usamos el método de Pandas para iterar por filas
        # Verificamos si el ID ya está en el conjunto de IDs únicos
        if fila['Id'] not in archivos_unicos:
            
            # Si el ID es único, agregamos la línea de entrada completa a la lista final
            dataframes.append(fila)
            # Agregamos el ID al conjunto de IDs únicos
            archivos_unicos.add(fila['Id'])

        # Extraemos los IDs y los agregamos al conjunto de IDs únicos
        archivos_unicos.update(set(df['Id']))

# Convertimos la lista de líneas únicas a un DataFrame
df_pelis_sin_repetir = pd.DataFrame(dataframes)

# Muestra el DataFrame sin la columna 'Género'
sin_genero = df_pelis_sin_repetir.drop(columns=['Género'])

# Restablecemos el índice para obtener una numeración consecutiva y sin saltos
sin_genero = sin_genero.reset_index(drop=True)

# Convertimos la lista de líneas únicas a un DataFrame
#df_pelis_sin_repetir = pd.DataFrame(dataframes)

# Restablecemos el índice para obtener una numeración consecutiva y sin saltos
#df_pelis_sin_repetir = df_pelis_sin_repetir.reset_index(drop=True)

#df_pelis_sin_repetir


sin_genero


In [24]:
# Usamos el método de Pandas para guardar un DataFrame en un archivo CSV
# Este argumento le indica a Pandas que no incluya el índice del DataFrame en el archivo CSV
sin_genero.to_csv('sin_genero.csv', index=False)
